In [9]:
import requests
import json

import pandas as pd

In [14]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


In [18]:
issues = [126, 122, 189, 153, 25, 158, 205, 270, 291,
      306, 96, 76, 118, 174, 311, 286, 199, 31, 115,
      64, 132, 46, 27, 106, 183, 50, 98, 190, 93,
      243, 266, 295, 231, 240, 244, 175, 232, 238,
      289, 230, 309, 308, 317, 218, 234, 298, 297,
      229, 233, 302, 265, 237, 59, 176, 301, 149, 58,
      134, 382, 217, 299, 239, 363, 360, 210, 359,
      379, 381, 34, 332, 293, 28, 378, 377, 383]

In [15]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:174) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [23]:
mod_query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:174) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

In [22]:
result = run_query(query)  # Execute the query

print(json.dumps(result['data'], indent=1))

{
 "repository": {
  "issue": {
   "number": 174,
   "title": "Clean air project (Theo Damoulas)",
   "url": "https://github.com/alan-turing-institute/Hut23/issues/174",
   "reactionGroups": [
    {
     "content": "THUMBS_UP",
     "users": {
      "edges": [
       {
        "node": {
         "login": "tomaslaz",
         "name": "Tomas Lazauskas"
        }
       },
       {
        "node": {
         "login": "thobson88",
         "name": "Tim Hobson"
        }
       },
       {
        "node": {
         "login": "nbarlowATI",
         "name": null
        }
       },
       {
        "node": {
         "login": "JimMadge",
         "name": "Jim Madge"
        }
       },
       {
        "node": {
         "login": "sgibson91",
         "name": "Sarah Gibson"
        }
       }
      ]
     }
    },
    {
     "content": "THUMBS_DOWN",
     "users": {
      "edges": [
       {
        "node": {
         "login": "Giovanni1085",
         "name": "Giovanni Colavizza"
        }
  

In [17]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',
          'LAUGH': '😄',
          'ROCKET': '🚀',
          'THUMBS_DOWN': '👎',
          'THUMBS_UP': '👍'}

In [34]:
result = run_query(mod_query)  # Execute the query

print(result['data']['repository']['issue']['title'])
for reaction_group in result['data']['repository']['issue']['reactionGroups']:
    print(emojis[reaction_group["content"]])
    for user in reaction_group["users"]["edges"]:
        if user["node"]["name"]:
            print(user["node"]["name"])
        else:
            print(user["node"]["login"])

Clean air project (Theo Damoulas)
👍
Tomas Lazauskas
Tim Hobson
nbarlowATI
Jim Madge
Sarah Gibson
👎
Giovanni Colavizza
myyong
😄
James Robinson
Evelina Gabasova
Louise Bowler
Camila Rangel Smith
miguelmorin
Jack Roberts
Oscar T Giles
Kasra Hosseini
🎉
😕
❤️
🚀
👀
